In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import talib as ta
import json
from pycoingecko import CoinGeckoAPI
from datetime import datetime as dt, timedelta as td

2021-08-25 21:23:21.638897: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-25 21:23:21.638928: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
config = {
    'id': 'cardano',
    'vs_curr': 'usd',
    'range': 1
}

In [3]:
now = dt.strptime(dt.now().strftime("%Y-%m-%d %H:%M:%S"),"%Y-%m-%d %H:%M:%S")
timestamp_list = [{'from': dt.timestamp(now - td(days=(x+1)*90)), 'to': dt.timestamp(now - td(days=x*90))} for x in reversed(range(config['range']))]

print(timestamp_list)

[{'from': 1622147004.0, 'to': 1629923004.0}]


In [4]:
cg = CoinGeckoAPI()
temp_closes = []

for timestamp in timestamp_list:
    mk_chart = cg.get_coin_market_chart_range_by_id(id=config['id'], vs_currency=config['vs_curr'], from_timestamp=timestamp['from'], to_timestamp=timestamp['to'])
    for i in range(len(mk_chart['prices'])):
        temp_closes.append(mk_chart['prices'][i][1])

closes = np.array(temp_closes)

print(len(closes))

2161


In [5]:
sma_7 = ta.ROC(ta.SMA(closes, 7), timeperiod=1)
sma_25 = ta.ROC(ta.SMA(closes, 25), timeperiod=1)
sma_99 = ta.ROC(ta.SMA(closes, 99), timeperiod=1)
ema_9 = ta.ROC(ta.EMA(closes, 9), timeperiod=1)
ema_26 = ta.ROC(ta.EMA(closes, 26), timeperiod=1)
bbands_up, bbands_mid, bbands_low = ta.BBANDS(closes, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
bbands_up = ta.ROC(bbands_up, timeperiod=1)
bbands_mid = ta.ROC(bbands_mid, timeperiod=1)
bbands_low = ta.ROC(bbands_low, timeperiod=1)
rsi = ta.RSI(closes, 14)
cmo = ta.CMO(closes, timeperiod=14)
macd, macdsignal, macdhist = ta.MACD(closes, fastperiod=12, slowperiod=26, signalperiod=9)
ppo = ta.PPO(closes, fastperiod=12, slowperiod=26, matype=0)
roc = ta.ROC(closes, timeperiod=1)
rocr = ta.ROCR(closes, timeperiod=1)
linearreg = ta.ROC(ta.LINEARREG(closes, timeperiod=14), timeperiod=1)
tsf = ta.ROC(ta.TSF(closes, timeperiod=14), timeperiod=1)

In [6]:
class_data = []

for n in ta.ROC(closes, timeperiod=1):
    if n < -3:
        class_data.append('V_LOW')
    elif -3 <= n <= 0:
        class_data.append('LOW')
    elif 0 < n <= 3:
        class_data.append('HIGH')
    else:
        class_data.append('V_HIGH')

In [7]:
data = [
    sma_7[-1],
    sma_25[-1],
    sma_99[-1],
    ema_9[-1],
    ema_26[-1],
    bbands_up[-1],
    bbands_mid[-1],
    bbands_low[-1],
    rsi[-1],
    cmo[-1],
    macd[-1],
    macdsignal[-1],
    macdhist[-1],
    ppo[-1],
    roc[-1],
    rocr[-1],
    linearreg[-1],
    tsf[-1]
]

df = [
    sma_7[2138:-1],
    sma_25[2138:-1],
    sma_99[2138:-1],
    ema_9[2138:-1],
    ema_26[2138:-1],
    bbands_up[2138:-1],
    bbands_mid[2138:-1],
    bbands_low[2138:-1],
    rsi[2138:-1],
    cmo[2138:-1],
    macd[2138:-1],
    macdsignal[2138:-1],
    macdhist[2138:-1],
    ppo[2138:-1],
    roc[2138:-1],
    rocr[2138:-1],
    linearreg[2138:-1],
    tsf[2138:-1],
    np.array(class_data[2139::])
]
# print(df[:,1][0:-1])
print([row[0] for row in df][0:-1])

[-0.30593225390241896, -0.31239347053846345, -0.08308840853957777, -0.2558507574552338, -0.2789594592992217, -0.16377397538690497, -0.3059896519898686, -0.4669341925046644, 33.87424903060466, -32.25150193879069, -0.005922676998446619, -0.0049817047493427085, -0.0009409722491039102, -2.8146085059065555, -1.5797716445210885, 0.9842022835547891, -0.13066749897572283, -0.07053034978212747]


In [8]:
# Outras coisas que não são relevantes para o projecto
#  - É possível guardar o modelo treinado e carregá-lo novamente
#    - Tem de ser guardado em dois ficheiros
#       - A estrutura em formato de JSON usando "model.to_json()" e gravando o modelo num ficheiro
#          - Para ler a estrutura lê-se o ficheiro e depois usa-se "tf.keras.models.model_from_json(json_savedModel)"
#       - Os pesos da rede usando o "model.save_weights('iris_weights.h5')"
#          - Para ler os pesos usa-se "model.load_weights('iris_weights.h5')"
#  - É possível fazer previsões usando "model.predict(np.array([[5.9, 3.0,	4.3, 1.5]]))"
#     - O resultado são logits que têm de ser convertidos em probabilidades usando "tf.nn.softmax(logits)"
#     - O predict só recebe as features, a label não é conhecida (é o que pretendemos saber)

# Estes elementos não são necessários para o projecto pois só necessitam dos valores para
# fazer gráficos e tabelas
# Caso tenham curiosidade sobre estes pontos extra falem com o docente

# json_model = model.to_json()

# with open("models/btc_model.json", "r") as outfile:
#     outfile.write(json_model)

# model.save_weights("models/" + symbol + "_weights.h5")

In [12]:
with open("models/ada_model.json", "r") as json_file:
    i = 3
    model = tf.keras.models.model_from_json(json_file.read())
    model.load_weights("models/ada_weights.h5")
    logits = model.predict(np.array([[row[i] for row in df][0:-1]]))
    
    print(tf.nn.softmax(logits))
    print([row[i] for row in df][-1])


tf.Tensor([[7.0210108e-19 9.3655008e-01 6.3449942e-02 4.8418994e-17]], shape=(1, 4), dtype=float32)
LOW
